In [2]:
# SYSTEM DEFAULT CODE, YOU DON'T NEED TO PAY ATTENTION

# 查看当前挂载的数据集目录, 该目录下的变更重启环境后会自动还原
# View dataset directory. This directory will be recovered automatically after resetting environment. 
!ls /home/aistudio/data

dataset.tsv


In [3]:
# SYSTEM DEFAULT CODE, YOU DON'T NEED TO PAY ATTENTION

# 查看工作区文件, 该目录下的变更将会持久保存. 请及时清理不必要的文件, 避免加载过慢.
# View personal work directory. All changes under this directory will be kept even after reset. Please clean unnecessary files in time to speed up environment loading.
!ls /home/aistudio/work

In [4]:
# SYSTEM DEFAULT CODE, YOU DON'T NEED TO PAY ATTENTION

# 如果需要进行持久化安装, 需要使用持久化路径, 如下方代码示例:
# If a persistence installation is required, you need to use the persistence path as the following:
!mkdir /home/aistudio/external-libraries
!pip install beautifulsoup4 -t /home/aistudio/external-libraries

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 122kB 12.6MB/s eta 0:00:01


In [5]:
# SYSTEM DEFAULT CODE, YOU DON'T NEED TO PAY ATTENTION

# 同时添加如下代码, 这样每次环境(kernel)启动的时候只要运行下方代码即可:
# Also add the following code, so that every time the environment (kernel) starts, just run the following code:
import sys
sys.path.append('/home/aistudio/external-libraries')

In [1]:
#Install paddlehub environment, must with version 1.8.2! Or there will be errors

!pip install paddlehub==1.8.2
import paddlehub as hub

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 337kB 13.7MB/s eta 0:00:01
  Found existing installation: paddlehub 2.0.4
    Uninstalling paddlehub-2.0.4:
      Successfully uninstalled paddlehub-2.0.4


In [2]:
import numpy as np
import pandas as pd
import json
import time
import re

In the next block, we load the pretrained BERT model. We select a slight model for convenience.

In [3]:

# Load BERT pretrained model
model=hub.Module(name="bert_chinese_L-12_H-768_A-12")

The next block will be the data preprocessing block.

In the next block, we do some data preprocessing. The training data is stored in a .json file named result.json (I know this name looks very misleading). The code in the block unpack the .json file, arrange them into ndarray objects: x_data and y_data, which corresponds to the texts and the tags.

Next, we convert the dataset to DataFrame objects and split them into train, validate and test set randomly.

We also notice that we have to save the dataset into .tsv files in the next section. To do this, we separate the characters using unseen \002 markers.

In [4]:
# data in result.json is the labeled ming data
with open('/home/aistudio/result.json') as jf:
    ming=json.load(jf)


#unpack .json file, wash the data and transfer them into ndarray
# x_data, y_data corresponds to texts and tags
def dataGen(ming):
    person_ids=np.array(list(ming.keys()))
    person_ids.sort()

    #idNum=len(person_ids)

    x_data=[]
    y_data=[]

    indexer=0
    for person_id in person_ids:
        
        char_tag=ming[person_id]['char_tag']
        x_data.append([])
        y_data.append([])
        omit_len=len(person_id)
        for i in range(omit_len+1,len(char_tag)):
            x_data[indexer].append(char_tag[i][0])
            y_data[indexer].append(char_tag[i][1])
         
        indexer=indexer+1

    for i in range(0,len(y_data)):
      for j in range(0,len(y_data[i])):
        old_text=y_data[i][j]
        #convert labels like 'B_date_reign' into 'B-date-reign'
        new_text=old_text.replace("_","-")
        y_data[i][j]=new_text


    return x_data,y_data,person_ids


#construct train,validate and test set
#train_set_rate indicates the proportion of trainning data
#validate_set_rate indicates the proportion of validation data
def splitTrain(x_data,y_data,person_ids,train_set_rate,validate_set_rate):
    x_data=np.array(x_data)
    y_data=np.array(y_data)
    
    temp=np.array([x_data,y_data])
    temp=temp.T
    
    ming_data=pd.DataFrame(temp,index=person_ids,columns=['text_a','label'])

    #To save data as .tsv files, we must separate the characters using \002 marker
    for i in range(0,len(ming_data['text_a'])):
        #print(i)
        ming_data['text_a'][i]='\002'.join(ming_data['text_a'][i])
        ming_data['label'][i]='\002'.join(ming_data['label'][i])
        ##ming_data['text_a'][i]+='\002'
        ##ming_data['label'][i]+='\002'
        #ming_data['text_a'][i]=str(ming_data['text_a'][i])
        #ming_data['label'][i]=str(ming_data['label'][i])
        
        

    np.random.seed(int(time.time()))
    ming_data=ming_data.sample(frac=1.0)
    
    idNum=len(person_ids)
    train_size=int(np.floor(idNum*train_set_rate))
    validate_size=int(np.floor(idNum*validate_set_rate))

    train_set=ming_data[0:train_size]

    validate_set=ming_data[train_size:train_size+validate_size]
    test_set=ming_data[train_size+validate_size:idNum]
    return train_set,validate_set,test_set,ming_data


#Build the first model: 50% of ming as train and 20% of ming as test
x_data,y_data,person_ids=dataGen(ming)
t,v,testing,ming_data=splitTrain(x_data,y_data,person_ids,0.75,0.25)

#generate x,y of train,validate and test
x_train=np.array(t['text_a'])
y_train=np.array(t['label'])
x_validate=np.array(v['text_a'])
y_validate=np.array(v['label'])

x_test=np.array(testing['text_a'])
y_test=np.array(testing['label'])

# We can save them as .csv files if we want
t.to_csv('train.csv')
v.to_csv('validate.csv')

# Acquire the length of training set
train_len=[]
for i in range(0,len(x_train)):
  train_len.append(len(x_train[i]))

max_len=max(train_len)

# The tag list, which specifies how many tags are of interest.
tag_list = tag_list = ["O",
        "B_date_reign", "I_date_reign",
        "B_date_year", "I_date_year",
        "B_office_voa", "I_office_voa",
        "B_office_title", "I_office_title",
        "B_place_placename", "I_place_placename"]

The next code block save our data to .tsv files.

In [5]:
# Part of Task1: save the data into tsv
# t-> train, v-> validate, testing -> test
# ming_data -> whole dataset, also saved in case it might be of use
t.to_csv('/home/aistudio/train.tsv',sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)
v.to_csv('/home/aistudio/validate.tsv',sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)
testing.to_csv('/home/aistudio/testing.tsv',sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)
ming_data.to_csv('/home/aistudio/data/dataset.tsv',sep='\t',columns=['text_a','label'],encoding='utf_8_sig',index=None)

# predict_data is the Jin biographical data without tags
predict_data=pd.read_table('/home/aistudio/test_data.txt')

# 'content_without_name' consists of characters in the texts
text=predict_data['content_without_name']

# Also save it as .tsv files in case
text.to_csv('/home/aistudio/predict.tsv',encoding='utf_8_sig')

In the next block, we load the dataset into the **DemoDataset** object.

In [8]:
from paddlehub.dataset.base_nlp_dataset import BaseNLPDataset

# Construct the dataset for task1
class DemoDataset(BaseNLPDataset):
    """DemoDataset"""
    def __init__(self):
        # Position where the data is stored
        self.dataset_dir = "/home/aistudio/"
        super(DemoDataset, self).__init__(
            base_path=self.dataset_dir,
            # train, validate and test
            train_file="train.tsv",
            dev_file="validate.tsv",
            test_file="testing.tsv",
            # If we have any predict dataset
            predict_file="predict.tsv",
            train_file_with_header=True,
            dev_file_with_header=True,
            test_file_with_header=True,
            predict_file_with_header=True,
            # the label maps. This list contain all tags in our training data
            label_list=["O",
        "B-date-reign", "I-date-reign",
        "B-date-year", "I-date-year",
        "B-office-voa", "I-office-voa",
        "B-office-title", "I-office-title",
        "B-place-placename", "I-place-placename"])

# load data
task1_dataset = DemoDataset()

In the next blocks, we define the training task. To define the training task, we shall first set up various configurations.

In [9]:
#Construct Reader to preprocess the data
reader = hub.reader.SequenceLabelReader(
        #specify the dataset (the task1_dataset we've just defined above)
        dataset=task1_dataset,
        vocab_path=model.get_vocab_path(),
        # Maximum sequence length, must be longer than any of the sequence
        # in train, validate, test and predict
        max_seq_len=512)

#Select the optimizaion strategy
strategy=hub.AdamWeightDecayStrategy(
    weight_decay=0.01,
    learning_rate=1e-4,
    warmup_proportion=0.1
)

# Define the running configurations. To be more specific, hyperparameters
config=hub.RunConfig(
    # When using GPUs, please turn this on
    use_cuda=True,
    # Number of Epochs
    num_epoch=80,
    # Specify a filename containing the checkpoint records
    # The training log will be saved in the filename specified
    checkpoint_dir="chinese_wwm_base_seq_label_demo",
    # Specify the batch size
    batch_size=16,
    # For how many intervals the model evaluate the F1, loss, recall, etc.
    # The evaluatio results are saved in checkpoint_dir
    eval_interval=50,
    strategy=strategy
)

In [10]:
#Construct fine tune task(for task 1)
inputs, outputs, program = model.context(
    trainable=True, max_seq_len=512)

sequence_output=outputs["sequence_output"]

feed_list=[
    inputs["input_ids"].name,
    inputs["position_ids"].name,
    inputs["segment_ids"].name,
    inputs["input_mask"].name
]

#Please again make sure that Paddlehub is of 1.8.2 version, or there will be error（parameter has no attribute...)
seq_label_task1 = hub.SequenceLabelTask(
    data_reader=reader,
    feature=sequence_output,
    feed_list=feed_list,
    max_seq_len=512,
    num_classes=task1_dataset.num_labels,
    config=config,
    # Add a layer of conditional random field
    add_crf=True)


Now we start the fine-tune task:

In [11]:

# Execute finetune task
task1_rate=seq_label_task1.finetune_and_eval()

Paddlehub provides APIs to directly predict the data. However, data passed to the predict API should be in the list format, with each of its element another list containing the biography texts. To be specific, the data to be predict should be in the format like this:

```py
to_predict=[['曾任宣差總管鷹房打捕東勝等處渡河船隻河道所。'],
['河內人，天會十四年參與修北村湯王廟。'],
['孟津人，進義校尉，大定四年任河南路同知兼知陜州。']]
```

So if the data to predict is kept as a DataFrame object in the program, you shall use the following code to transform it into the format above:

In [14]:

#Execute predict task
#import test_data to predict

predict=[]
for i in range(0,len(predict_data['content_without_name'])):
    predict.append([predict_data['content_without_name'][i]])

# Just to make sure it is really done :)
print('done')

#Rubbish code, ignore it
#pred=seq_label_task_1.predict(data=predict_data['content_without_name'])

done


And here comes the prediction:

In [16]:
#predict
pred=seq_label_task1.predict(data=predict)

The next thing to do is to label the texts with the predict result. However, if you call the pred object, you will find that it is a list consists of addresses in the memory. For each address, if you call 'pred[i].run_results', you still get an integer instead of a specific tag. Actually, the integers involved corresponds to the tags (Like B-Place-Name, O, I-Reign, etc). So we have to convert these integers into tags to make them readable. This is not a very easy task.

We first acquire the results in integer form and acquire the **results** object.

If we call the results object, we will find that it is in a strange format. For each element in results, it consists of two arrays. The first array is a list of integers varying from 0-10 and the second array is a lists of positive integers. Let me explain why this will happen.

The predict results are returned in batches. Each element in the results list is a batch of predictions. The first array in a elements are tagged results for all characters in this batch encoded in integers, while the second array encodes the number of biographies in each batch (the length of the 2nd array) and the length of each biography (the integers in the 2nd array). Hence, to reconstruct the results, we split the first array in each batch according to the lengths encoded in the 2nd array and use the inv_label_map to transform them into tags.

In [14]:
# The output of paddlehub is written in a strange order
# The following codes convert the output labels into readable tags

# Acquire results that are shown as integers
results=[p.run_results for p in pred]

# The inverse label map that identifies which tag each integer shall correspond to
# The reason why we know the mapping rule is that it follows the order of the tags in label_map
# dictionary mentioned above
inv_label_map={0:'O',1:'B-date-reign',2:'I-date-reign',3:'B-date-year',4:'I-date-year',
5:'B-office-voa',6:'I-office-voa',7:'B-office-title',8:'I-office-title',9:'B-place-placename',
10:'I-place-placename'}

# Initialize the tags
tags=[]

# Transform integers to tags
for num_batch, batch_results in enumerate(results):
    infers = batch_results[0].reshape([-1]).astype(np.int32).tolist()
    
    #acquire the length of each text in batch #num_batch
    np_lens = batch_results[1]
    
    
    vernier=0

    for index, np_len in enumerate(np_lens):
        
        #labels = infers[index * 400:(index + 1) * 400]
        labels=infers[vernier:vernier+np_len]
        vernier=vernier+np_len

        label_str = []
        count = 0
        for label_val in labels:
            label_str.append(inv_label_map[label_val])
            count += 1
            if count == np_len-1:
                break
        tags.append(label_str)


# the first position of each prediction is a marker, throw away
for i in range(0,len(tags)):
    #print(i)
    tags[i]=tags[i][1:len(tags[i])]

To make it looks more COOOL, we save the output as .txt files along with the biographies. Now you can read the biographies with tagged results 

In [16]:
text=[]

#Write predicted tags into .txt files

for i in range(0,len(predict)):
    text.append(predict[i][0])

data={'text':text,'tags_predicted':tags}

frame=pd.DataFrame(data)

for i in range(0,len(text)):
    print(i)
    ner_result={'char':list(text[i]),'tag':list(tags[i])}
    frame=pd.DataFrame(ner_result,columns=['char','tag'])
    frame.to_csv('/home/aistudio/jin_ner_0203/'+str(bio_ids[i])+'.txt',sep=" ")

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 